Modèles à tester : 
- Régression linéaire
- SGD
- DNN

In [1]:
# Chargement des données
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

enc = OneHotEncoder(handle_unknown="ignore")

df = pd.read_csv("./true_car_listings.csv").sample(n=300000)
df["state"] = df["state"].str.strip()
df["make"] = df["make"].str.strip()
df["model"] = df["model"].str.strip()
cat = df[["state", "make", "model"]]
enc.fit(cat)

with open("features_encoder.pkl", "wb") as encoder_file:
    pickle.dump(enc, encoder_file)

one_hot_vec = enc.transform(cat).toarray()

X = np.hstack((one_hot_vec, df["year"].values[:, None], df["mileage"].values[:, None]))
print(X.shape)
y = df["price"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


(300000, 2547)


In [23]:
# Test du SGDRegressor
model = SGDRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
sgd_mae = mean_absolute_error(y_test, y_pred)
print(f"MAE avec le SGD : {sgd_mae}")

MAE avec le SGD : 6.589579812110103e+17


In [34]:
# Test avec un réseau de neurones
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='loss', patience=3)

# define the keras model
model = Sequential()
model.add(Dense(100, input_shape=(X.shape[1],), activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs=150, batch_size=100, callbacks=[callback])
dnn_mae = model.evaluate(X_test, y_test)[0]

Epoch 1/150
1340/1340 [==============================] - 7s 5ms/step - loss: 8685.3486 - mean_absolute_percentage_error: 42.6253
Epoch 2/150
1340/1340 [==============================] - 6s 4ms/step - loss: 7902.3359 - mean_absolute_percentage_error: 38.7273
Epoch 3/150
1340/1340 [==============================] - 6s 4ms/step - loss: 7848.4287 - mean_absolute_percentage_error: 38.4163
Epoch 4/150
1340/1340 [==============================] - 6s 4ms/step - loss: 7790.5034 - mean_absolute_percentage_error: 38.0898
Epoch 5/150
1340/1340 [==============================] - 6s 4ms/step - loss: 7719.5410 - mean_absolute_percentage_error: 37.8598
Epoch 6/150
1340/1340 [==============================] - 6s 4ms/step - loss: 7433.1187 - mean_absolute_percentage_error: 36.2245
Epoch 7/150
1340/1340 [==============================] - 6s 4ms/step - loss: 6655.7402 - mean_absolute_percentage_error: 31.9924
Epoch 8/150
1340/1340 [==============================] - 6s 5ms/step - loss: 6083.1528 - mean_abs

In [32]:
print(f"MAE avec le DNN : {dnn_mae}")

MAE avec le DNN : 3530.277587890625
